writing my own code (COPY-PASTING CODE NOT ALLOWED, CODE IS OK) for cs231n's classification notes

this notebook is for: https://cs231n.github.io/linear-classify/

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace

In [4]:
from utils import load_cifar
Xtr_raw, Ytr, Xte_raw, Yte = load_cifar()

In [25]:
# adding biases
def add_bias(X): return np.hstack([X, np.ones((X.shape[0], 1))])
def preprocess(X): return (X - X.mean()) / (X.max() - X.min())
Xtr, Xte = map(add_bias, map(preprocess, [Xtr_raw, Xte_raw]))

In [29]:
labels = list(set(np.unique(Ytr)).union(set(np.unique(Yte))))
m, d = Xtr.shape
ncl = len(labels)
W = np.random.randn(ncl, d) * 1e-4

In [32]:
class Classifier:
    def __init__(self, delta=1):
        self.delta = delta

    def L_i(self, x, y, W):
        """
        unvectorized version. Compute the multiclass svm loss for a single example (x,y)
        - x is a column vector representing an image (e.g. 3073 x 1 in CIFAR-10)
            with an appended bias dimension in the 3073-rd position (i.e. bias trick)
        - y is an integer giving index of correct class (e.g. between 0 and 9 in CIFAR-10)
        - W is the weight matrix (e.g. 10 x 3073 in CIFAR-10)
        """
        score = np.dot(W, x)
        s = 0
        for j in range(W.shape[0]):
            if j != y:
                s += max(0, score[j] - score[y] + self.delta)
        return s

    def L_i_vectorized(self, x, y, W):
        """
        A faster half-vectorized implementation. half-vectorized
        refers to the fact that for a single example the implementation contains
        no for loops, but there is still one loop over the examples (outside this function)
        """
        score = np.dot(W, x)
        return np.sum(np.maximum(0, score - score[y] + self.delta)) - self.delta

    def L(self, X, y, W):
        """
        fully-vectorized implementation :
        - X holds all the training examples as columns (e.g. 3073 x 50,000 in CIFAR-10)
        - y is array of integers specifying correct class (e.g. 50,000-D array)
        - W are weights (e.g. 10 x 3073)
        """
        score = np.dot(W, X)
        score_lab = score[y, np.arange(X.shape[1])]
        return (np.sum(np.maximum(0, score - score_lab + self.delta)) - self.delta * y.shape[0]) / X.shape[1]

In [33]:
clf = Classifier()
print(f"basic: {clf.L_i(Xtr[0], Ytr[0], W)}")
print(f"semi vectorized: {clf.L_i_vectorized(Xtr[0], Ytr[0], W)}")
print(f"vectorized: {clf.L(Xtr.T, Ytr, W)}")

basic: 9.0129304290142
semi vectorized: 9.0129304290142
vectorized: 9.000471991899438
